In [48]:
import os
import sys
import pandas as pd 
import numpy as np
from data_import import *
from phenotype_correlation import *
from genotype_correlation import *
from baseline_mixed_model import *
import statsmodels.api as sm

# import data
geno_df, pheno_df = import_data_svenson()

Samples:
['F326', 'F327', 'F329', 'F330', 'F331', 'F332', 'F333', 'F334', 'F335', 'F336', 'F337', 'F338', 'F339', 'F340', 'F341', 'F342', 'F343', 'F344', 'F345', 'F346', 'F347', 'F348', 'F349', 'F350', 'F351', 'F352', 'F353', 'F354', 'F355', 'F356', 'F357', 'F358', 'F359', 'F360', 'F361', 'F362', 'F363', 'F364', 'F365', 'F366', 'F367', 'F368', 'F369', 'F371', 'F372', 'F373', 'F374', 'F375', 'F376', 'F377', 'F378', 'F379', 'F380', 'F381', 'F382', 'F383', 'F384', 'F385', 'F386', 'F387', 'F388', 'F389', 'F390', 'F391', 'F392', 'F393', 'F394', 'F395', 'F396', 'F397', 'F398', 'F399', 'F400', 'F401', 'F402', 'F403', 'F405', 'F406', 'F407', 'F408', 'F410', 'F411', 'F412', 'F413', 'F414', 'F415', 'F416', 'F417', 'F418', 'F419', 'F420', 'F421', 'F422', 'F423', 'F424', 'F425', 'M326', 'M327', 'M328', 'M329', 'M330', 'M331', 'M332', 'M333', 'M334', 'M335', 'M336', 'M337', 'M339', 'M340', 'M341', 'M342', 'M343', 'M344', 'M345', 'M347', 'M348', 'M349', 'M350', 'M351', 'M352', 'M353', 'M354', 'M355'

/Users/harryyang/Documents/Research/Class/CS229/cm-229/data_import.py:73: RuntimeWarning: Mean of empty slice
  np.nanmean(geno_df_select_transpose, axis=0))/np.nanstd(geno_df_select_transpose, axis=0)
/Users/harryyang/anaconda3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [49]:
# testing with ghrelin and leptin 
phenotype_1 = "weight1"
phenotype_2 = "length1"
phenotype_3 = "fat_mri"
missing_rate = 0.1
verbose = True
sample_list = list()

In [93]:
corr_mat = calculate_highly_correlated_phenotypes(pheno_df)

print("The correlation between %s and %s is %f" % (phenotype_1, phenotype_2, corr_mat[phenotype_1][phenotype_2]))

# bind phenotype into list to extract
phenotype_list = [phenotype_1, phenotype_2]

# extract the phenotypes 
geno_select, pheno_select = select_phenotype_multiple_phenotypes(geno_df, pheno_df, phenotype_list = phenotype_list, verbose = verbose)

# separate training and test dataset 
geno_tr, pheno_tr, geno_test, pheno_test, test_sample_list = separate_training_test(geno_select, pheno_select, missing_rate = missing_rate, sample_list_select = sample_list)

The correlation between weight1 and length1 is 0.819920
These samples are missing phenotypes:
[]
(72903, 187)
(4887, 187)
(187, 2)


In [94]:
# perform simple ridge to identify the top SNPs 
lm_re = sm.OLS(endog = pheno_tr[phenotype_2], exog = geno_tr.transpose()).fit_regularized(L1_wt = 1.0)

if verbose: 
    print(lm_re.params)

UNC010515443          0.595000
UNC16386              1.534974
UNC34218              0.911665
UNC49759             -0.235371
UNC010005983          0.219446
JAX00241357          -0.117977
UNC70761             -0.301386
backupJAX00000484    -0.255459
UNC90639              0.064624
JAX00241997           0.568197
UNC130697             0.227215
JAX00242455           0.405715
UNC159456             0.926010
UNC159803            -0.225193
UNC162008            -0.249763
UNC177767             0.297114
UNC179038            -0.067679
UNC187637             0.696899
UNC189115             0.158569
UNC228500            -0.298160
UNC243187             0.310750
UNC244201            -0.064721
backupUNC010527629    0.246150
UNC249229            -0.118094
UNC251384            -0.299205
JAX00244193           0.024580
UNC271472             0.274370
UNC273653            -0.275397
UNC278071             0.151494
UNC282212            -0.535411
                        ...   
UNC31001485          -0.329309
JAX00714

In [95]:
# select those genotypes only for select run
top_N = 10
top_N_idx = np.argsort(abs(lm_re.params))[-top_N:]

top_N_values = [lm_re.params[i] for i in top_N_idx]

top_N_snps = geno_tr.iloc[top_N_idx].index
top_N_snps

Index(['JAX00508001', 'UNC12275646', 'JAX00497684', 'UNC34218', 'UNC159456',
       'UNC8554223', 'UNC6698408', 'UNC10494469', 'UNC8513612', 'UNC16386'],
      dtype='object', name='marker')

In [96]:
# perform OLS 
lm = sm.OLS(endog = pheno_tr[phenotype_2], exog = pheno_tr[phenotype_1]).fit()

if verbose:	
    print("The linear model summary for predicting phenotype %a based on phenotype %a" % (phenotype_2, phenotype_1))
    print(lm.summary())	
    print(lm.params)	

# prediction for fixed effect
predictions_fe = lm.predict(pheno_test[phenotype_1])

The linear model summary for predicting phenotype 'length1' based on phenotype 'weight1'
                            OLS Regression Results                            
Dep. Variable:                length1   R-squared:                       0.972
Model:                            OLS   Adj. R-squared:                  0.972
Method:                 Least Squares   F-statistic:                     5944.
Date:                Thu, 13 Jun 2019   Prob (F-statistic):          3.49e-134
Time:                        11:12:24   Log-Likelihood:                -309.01
No. Observations:                 171   AIC:                             620.0
Df Residuals:                     170   BIC:                             623.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------

In [92]:
# perform ridge regression on the residual (random effect part)
residuals = pheno_tr[phenotype_2] - lm.predict(pheno_tr[phenotype_1])

lm_re = sm.OLS(endog = residuals, exog = geno_tr.loc[top_N_snps].transpose()).fit_regularized(L1_wt = 1.0)

if verbose: 
    print(lm_re.params)

predictions_re = lm_re.predict(geno_test.loc[top_N_snps].transpose())

# combine the result from both
total_prediction = predictions_fe + predictions_re

mse = calculate_MSE(total_prediction, pheno_test[phenotype_2])
mse
#return(mse, test_sample_list)

UNC11734899    0.065062
UNC13856737    0.290376
UNC13760792   -0.036665
UNC29627248    0.125451
UNC16274148   -0.277348
UNC6772490     0.111500
JAX00065782   -0.408705
JAX00548019   -0.164901
UNC31514370   -0.200480
UNC31156076   -0.116193
dtype: float64


3.1615950499824326

In [87]:
# perform ridge regression on the residual (random effect part)
residuals = pheno_tr[phenotype_2] - lm.predict(pheno_tr[phenotype_1])

lm_re = sm.OLS(endog = residuals, exog = geno_tr.transpose()).fit_regularized(L1_wt = 0.0)

if verbose: 
    print(lm_re.params)

predictions_re = lm_re.predict(geno_test.transpose())


[ 0.00400049  0.00822862 -0.00702599 ... -0.01295389  0.00247644
 -0.01272774]


In [88]:
# combine the result from both
total_prediction = predictions_fe + predictions_re

mse = calculate_MSE(total_prediction, pheno_test[phenotype_2])

#return(mse, test_sample_list)

In [89]:
mse

14.165291908852646